In [1]:
import requests
import bs4
from datetime import datetime
import re
import os
import urllib
import urllib2
from bs4 import BeautifulSoup
from selenium import webdriver
import time

initialpage = 'https://www.boxofficemojo.com/yearly/chart/?yr=2017&p=.htm'
res = requests.get(initialpage, timeout=None)
soup = bs4.BeautifulSoup(res.text, 'html.parser')


In [2]:
pages = []
pagelinks=soup.select('a[href^="/yearly/chart/?page"]')

for i in range(int(len(pagelinks)/2)):
    pages.append(str(pagelinks[i])[9:-14])
    pages[i]=pages[i].replace("amp;","")
    pages[i]= "https://www.boxofficemojo.com" + pages[i]
    pages[i]=pages[i][:-1]
       
    
    
pages.insert(0, initialpage)
date_dic = {}
movie_links = []
titles = []
Domestic_Gross_Arr=[]
Genre_Arr=[]
Release_Date_Arr = []
Theaters_Arr=[]
Budget_Arr = []
Views_Arr = []
Edits_Arr = []
Editors_Arr = []


for i in range(int(len(pagelinks)/2 + 1)): 
    movie_count=0;
    res1 = requests.get(pages[i])
    souppage=bs4.BeautifulSoup(res1.text, 'html.parser')
    for j in souppage.select('tr > td > b > font > a'):
        link = j.get("href")[7:].split("&")
        str1 = "".join(link)
        final = "https://www.boxofficemojo.com/movies" + str1
        if "/?id" in final: 
            movie_links.append(final)
            movie_count += 1
            
    number_of_theaters=souppage.find("tr", bgcolor="#dcdcdc")
    for k in range(movie_count):movie_count
        print(number_of_theaters.next_sibling.contents[4].text)
        Theaters_Arr.append(number_of_theaters.next_sibling.contents[4].text)
        number_of_theaters=number_of_theaters.next_sibling
        
  

4,232
4,210
1,186
1,803
1,070
380
705
557
13
7
2
10
3
1
2
2


In [3]:
k=0
path = os.getcwd()  
path = path + '/movie_pictures'
os.makedirs(path)
os.chdir(path)
while(k < len(titles)):
    j = movie_links[k]
    try:
        res1 = requests.get(j)
        soup1 = bs4.BeautifulSoup(res1.text, 'html.parser')
    
        c = soup1.select('td[width="35%"]')
        d=soup1.select('div[class="mp_box_content"]')
   
        genre = soup1.select('td[valign="top"]')[5].select('b')
        image = soup1.select('img')[6].get('src')
        budget = soup1.select('tr > td > b')
        domestic = str(c[0].select('b'))[4:-5]
    
   
        release = soup1.nobr.a
        title = soup1.select('title')[0].getText()[:-25]
        print ("-----------------------------------------")
        print ("Title: " +title)
        titles.append(title)
        print ("Domestic Gross: " +domestic)
        Domestic_Gross_Arr.append(domestic)
        print ("Genre: "+genre[0].getText())
        Genre_Arr.append(genre[0].getText())
        print ("Release Date: " +release.contents[0])
        Release_Date_Arr.append(release.contents[0])
        print ("Production Budget: " +budget[5].getText())
        Budget_Arr.append(budget[5].getText())
        
        year1=str(release.contents[0])[-4:]
        a,b=str(release.contents[0]).split(",")
        month1, day1=a.split(" ")
        datez= year1 + month1 + day1
        new_date= datetime.strptime(datez , "%Y%B%d")
        date_dic[title]=new_date       
    
    
        with open('pic' + str(k) + '.png', 'wb') as handle:
            response = requests.get(image, stream=True)
        
            if not response.ok:
                print response

            for block in response.iter_content(1024):
                if not block:
                    break

            handle.write(block)
    except:
        print("Error Occured, Page Or Data Not Available")
    k+=1

-----------------------------------------
Title: Star Wars: The Last Jedi
Domestic Gross: $620,181,382
Genre: Sci-Fi Fantasy
Release Date: December 15, 2017
Production Budget: N/A
-----------------------------------------
Title: Beauty and the Beast (2017)
Domestic Gross: $504,014,165
Genre: Fantasy
Release Date: March 17, 2017
Production Budget: $160 million


In [4]:
def subtract_one_month(t):

    import datetime
    one_day = datetime.timedelta(days=1)
    one_month_earlier = t - one_day
    while one_month_earlier.month == t.month or one_month_earlier.day > t.day:
        one_month_earlier -= one_day
    year=str(one_month_earlier)[:4]

    day=str(one_month_earlier)[8:10]
   
    month=str(one_month_earlier)[5:7]

    newdate= year + "-" + month +"-" + day
 
    return newdate

In [5]:
number_of_errors=0
browser = webdriver.Chrome("/Users/Gokce/Downloads/chromedriver")
browser.maximize_window() 
browser.implicitly_wait(20)
for i in titles:
    try:
        release_date = date_dic[i]
        i = i.replace(' ', '_')
        i = i.replace("2017", "2017_film")
    #end = datetime.strptime(release_date, '%B %d, %Y')


        end_date = release_date.strftime('%Y-%m-%d')
        start_date = subtract_one_month(release_date)
        url = "https://tools.wmflabs.org/pageviews/?project=en.wikipedia.org&platform=all-access&agent=user&start="+ start_date +"&end="+ end_date + "&pages=" + i

    
        browser.get(url)
        page_views_count = browser.find_element_by_css_selector(" .summary-column--container .legend-block--pageviews .linear-legend--counts:first-child span.pull-right ")
        page_edits_count = browser.find_element_by_css_selector(" .summary-column--container .legend-block--revisions .linear-legend--counts:first-child span.pull-right ")
        page_editors_count = browser.find_element_by_css_selector(" .summary-column--container .legend-block--revisions .legend-block--body .linear-legend--counts:nth-child(2) span.pull-right ")
        print (i)
    
        print ("Number of Page Views: " +page_views_count.text)
        Views_Arr.append(page_views_count.text)
        print ("Number of Edits: " +page_edits_count.text)
        Edits_Arr.append(page_edits_count.text)
        print ("Number of Editors: " +page_editors_count.text)
        Editors_Arr.append(page_editors_count.text)
    except:
        print("Error Occured for this page: " + str(i))
        number_of_errors += 1
        Views_Arr.append(-1)
        Edits_Arr.append(-1)
        Editors_Arr.append(-1)
    
#.text
time.sleep(5)
browser.quit()


Star_Wars:_The_Last_Jedi
Number of Page Views: 3,286,079
Number of Edits: 443
Number of Editors: 103
Beauty_and_the_Beast_(2017_film)
Number of Page Views: 1,375,282
Number of Edits: 671
Number of Editors: 240


In [12]:
from os.path import expanduser
home = expanduser("~")

os.chdir( home + '/Desktop')
import pandas as pd


# Create a Pandas dataframe from some data.
a = {'Title': titles, 'Genre': Genre_Arr, 'Budget': Budget_Arr, 'Domestic_Gross': Domestic_Gross_Arr, 'Release_Date': Release_Date_Arr, 'Theaters': Theaters_Arr, 'Views': Views_Arr, 'Edits': Edits_Arr, 'Editors': Editors_Arr}

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')
#rowTitles = ['Title', 'Genre', 'Budget', 'Domestic_Gross', 'Release_Date', 'Theaters', 'Views', 'Edits', 'Editors']
#df.reindex(rowTitles)

df = pd.DataFrame.from_dict(a, orient='index')

df.transpose()

b, c = df.iloc[0], df.iloc[1]

temp = df.iloc[0].copy()
df.iloc[0] = c
df.iloc[1] = temp

df.to_excel(writer, sheet_name='Sheet1')



writer.save()

